In [7]:
def genData(data):
    newd = []
    for i in data:
        newd.append(format(ord(i), '08b'))
    return newd

In [8]:
import random
def create_sequence(size, key):
    sequence = []
    rnd = random.Random(key)
    for i in range(size):
        sequence.append(rnd.randint(1, 4))
    return sequence

In [9]:
def modPix(pix, data, key):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)
    seq = create_sequence(len(pix), key)
    idx = 0

    for i in range(lendata):
        pix = [value for value in imdata.__next__()[:3] + imdata.__next__()[:3] + imdata.__next__()[:3]]
        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j]% 2 != 0):
                pix[j] -= seq[idx]
            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if(pix[j] != 0):
                    pix[j] += seq[idx]
                else:
                    pix[j] += seq[idx] + 1
            idx = (idx + 1) % len(seq)
        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if(pix[-1] != 0):
                    pix[-1] -= seq[idx]
                else:
                    pix[-1] += seq[idx] + 1
            else:
                pix[-1] += seq[idx]
        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= seq[idx]
            else:
                pix[-1] += seq[idx]
        pix = tuple(pix)
        yield pix[0:3]
        yield pix[3:6]
        yield pix[6:9]


In [10]:
from PIL import Image

def encode(image,data, key):
    image = Imgae.open(image)
    #newimg = image.copy()
    
    #encode_enc(newimg, data, key)
    # res = str(hashlib.md5((newimg).tobytes()).hexdigest())
    # tx_hash = md.contract.functions.setPayload(res).transact()
    # tx_hash = 'tx_hash: {}'.format(tx_hash.hex())
    return newimg



In [11]:
def xor(a, b):
    # Perform XOR operation on two binary strings of equal length
    # and return the resulting binary string.
    return ''.join(str(int(x) ^ int(y)) for x, y in zip(a, b))

def decode(image, key):
    data = ''
    imgdata = iter(image.getdata())
    spread_seq = create_sequence(len(image.getdata()),key)
    while (True):
        pixels = [value for value in imgdata.__next__()[:3] + imgdata.__next__()[:3] + imgdata.__next__()[:3]]
        binstr = ''
        for i in pixels[:8]:
            if (i % 2 == 0):
                binstr += '0'
            else:
                binstr += '1'
        binstr = xor(binstr, spread_seq[:len(binstr)])
        spread_seq = spread_seq[len(binstr):]
        data += chr(int(binstr, 2))
        if (pixels[-1] % 2 != 0):
            return data
